In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
import json
import traceback
import datetime
import random
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

In [2]:
# Number of nodes in the cluster.
num_nodes=21

# Give a cluster name
slice_name='FL_High1'

# Site name, pick one site from the above list of resources.
sites = ['CLEM','DALL', 'FIU','STAR','UTAH','LOSA','MICH', 'NEWY','RUTG','PRIN','KANS','SEAT','HAWI','TOKY','CERN','UTAH','EDUKY','UCSD','GATECH','KANS','GPN']

# LEGACY VARIABLES
master_gpu=False

# Operation system, Linux distribution e.g. default_ubuntu_18, default_ubuntu_20, etc.
image='default_ubuntu_20'

In [3]:
node_names=[]
nic_names=[]
iface_names=[]
network_names=[]


for i in range(1,num_nodes+1):
    node_names.append("node{0}".format(i))
    nic_names.append("nic{0}".format(i))
    iface_names.append("iface{0}".format(i))
    network_names.append("net{0}".format(i))

print(node_names)
print(nic_names)
print(iface_names)

['node1', 'node2', 'node3', 'node4', 'node5', 'node6', 'node7', 'node8', 'node9', 'node10', 'node11', 'node12', 'node13', 'node14', 'node15', 'node16', 'node17', 'node18', 'node19', 'node20', 'node21']
['nic1', 'nic2', 'nic3', 'nic4', 'nic5', 'nic6', 'nic7', 'nic8', 'nic9', 'nic10', 'nic11', 'nic12', 'nic13', 'nic14', 'nic15', 'nic16', 'nic17', 'nic18', 'nic19', 'nic20', 'nic21']
['iface1', 'iface2', 'iface3', 'iface4', 'iface5', 'iface6', 'iface7', 'iface8', 'iface9', 'iface10', 'iface11', 'iface12', 'iface13', 'iface14', 'iface15', 'iface16', 'iface17', 'iface18', 'iface19', 'iface20', 'iface21']


In [105]:
try:
    slice = fablib.new_slice(name=slice_name)

    # Use dictionaries to store nodes, interfaces, and networks
    nodes = {}
    interfaces = {}
    networks = {}

    # Create networks
    for i in range(1, num_nodes + 1):
        networks[i] = slice.add_l3network(name=network_names[i-1], type='IPv4')

    # Create nodes and configure them
    for i in range(1, num_nodes + 1):
        node_name = node_names[i-1]  # Ensure correct indexing
        nodes[i] = slice.add_node(name=node_name, site=sites[i-1],cores=24,
                                ram=48,
                                disk=300,
                                image=image)

        # Add NIC component
        interfaces[i] = nodes[i].add_component(model='NIC_Basic', name=f'nic{i}').get_interfaces()[0]
        interfaces[i].set_mode('auto')

        # Attach NIC to the network
        networks[i].add_interface(interfaces[i])

        # Add route for the node
        nodes[i].add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=networks[i].get_gateway())

    # Submit the slice request
    slice.submit()

except Exception as e:
    print(f'Exception: {e}')



Retry: 18, Time: 813 sec


ID,9f7736a7-3107-4760-b192-95874bbeda87
Name,FL_High1
Lease Expiration (UTC),2025-04-24 23:27:28 +0000
Lease Start (UTC),2025-04-23 23:27:28 +0000
Project ID,b3cffedd-ddb4-43ee-b57d-459b768e14ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
d2fdca2e-4c74-43ef-93bf-2ec198b2f10d,node1,24,64,500,default_ubuntu_20,qcow2,clem-w2.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe7a:9137,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe7a:9137,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8d5f9b42-d820-4f43-9190-8be7d5f81638,node10,24,64,500,default_ubuntu_20,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fe19:418d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe19:418d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
38028455-a1c3-40b7-8f22-ceaa30486348,node11,24,64,500,default_ubuntu_20,qcow2,kans-w2.fabric-testbed.net,KANS,ubuntu,2001:400:a100:3060:f816:3eff:fe46:778d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fe46:778d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c7e05851-f5e7-435d-8ef5-de9c74994b8b,node12,24,64,500,default_ubuntu_20,qcow2,seat-w2.fabric-testbed.net,SEAT,ubuntu,2001:400:a100:3080:f816:3eff:fe62:d4ea,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe62:d4ea,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
93884021-683e-410c-977c-e7b2f39c53cd,node13,24,64,500,default_ubuntu_20,qcow2,hawi-w1.fabric-testbed.net,HAWI,ubuntu,2607:f278:1:202:f816:3eff:fea4:e2af,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f278:1:202:f816:3eff:fea4:e2af,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7ee24723-6d0e-4848-a6f6-e49bfda8cf0e,node14,24,64,500,default_ubuntu_20,qcow2,toky-w2.fabric-testbed.net,TOKY,ubuntu,133.69.160.18,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.18,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
31419306-4cfb-472a-a496-d0c1cb5b0317,node15,24,64,500,default_ubuntu_20,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe14:5147,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe14:5147,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2469a567-2e9a-47bb-a188-3fcf0fb7643c,node16,24,64,500,default_ubuntu_20,qcow2,utah-w5.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fea2:3e1e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fea2:3e1e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
69070002-7adc-4a45-b77f-d46499b81049,node17,24,64,500,default_ubuntu_20,qcow2,eduky-w7.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe2e:d4b0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe2e:d4b0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1e15cd6a-ffce-4cac-8467-f260bd42dd35,node18,24,64,500,default_ubuntu_20,qcow2,ucsd-w5.fabric-testbed.net,UCSD,ubuntu,132.249.252.143,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.143,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
2b1bdcd1-64d7-4cd6-84e4-525c2a04298c,net1,L3,FABNetv4,CLEM,10.136.1.0/24,10.136.1.1,Active,
49c00b81-62e6-4074-bc7f-e4126fa66b28,net10,L3,FABNetv4,PRIN,10.139.133.0/24,10.139.133.1,Active,
f1486016-7459-4f40-96c4-645c66ce9801,net11,L3,FABNetv4,KANS,10.138.10.0/24,10.138.10.1,Active,
1235c930-106d-4330-9ff6-d3ab7c216bab,net12,L3,FABNetv4,SEAT,10.139.3.0/24,10.139.3.1,Active,
94aebaa6-811b-4a8b-8f8f-be52ace534e7,net13,L3,FABNetv4,HAWI,10.147.5.0/24,10.147.5.1,Active,
21b5a742-c419-43e5-9a99-d36bc0284642,net14,L3,FABNetv4,TOKY,10.146.1.0/24,10.146.1.1,Active,
3f5df975-270d-484b-89a1-29ec2da07ef1,net15,L3,FABNetv4,CERN,10.143.8.0/24,10.143.8.1,Active,
7db5d96d-ba8e-4386-bfc6-7dfe3ac43fd2,net16,L3,FABNetv4,UTAH,10.132.13.0/24,10.132.13.1,Active,
9683856a-1e53-472a-9a4b-2446f4eec211,net17,L3,FABNetv4,EDUKY,10.147.129.0/24,10.147.129.1,Active,
745a200d-de5d-4165-8b92-d8fa01ba2c12,net18,L3,FABNetv4,UCSD,10.134.140.0/24,10.134.140.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
node1-nic1-p1,p1,node1,net1,100,auto,,02:DA:D0:DE:84:4D,enp7s0,enp7s0,10.136.1.2,4,HundredGigE0/0/0/9
node2-nic2-p1,p1,node2,net2,100,auto,,0E:0A:CD:51:48:0A,enp7s0,enp7s0,10.133.134.2,6,HundredGigE0/0/0/5
node3-nic3-p1,p1,node3,net3,100,auto,,0E:C7:B8:E0:CD:11,enp7s0,enp7s0,10.135.133.2,6,HundredGigE0/0/0/5
node4-nic4-p1,p1,node4,net4,100,auto,,22:16:D2:54:E0:CB,enp7s0,enp7s0,10.129.139.2,6,HundredGigE0/0/0/5
node5-nic5-p1,p1,node5,net5,100,auto,,16:4A:C8:88:50:28,enp7s0,enp7s0,10.132.5.2,4,HundredGigE0/0/0/11
node6-nic6-p1,p1,node6,net6,100,auto,,06:F9:BA:9E:5A:38,enp6s0,enp6s0,10.137.4.2,4,HundredGigE0/0/0/7
node7-nic7-p1,p1,node7,net7,100,auto,,0E:06:73:C1:D9:AE,enp7s0,enp7s0,10.131.9.2,6,HundredGigE0/0/0/5
node8-nic8-p1,p1,node8,net8,100,auto,,06:AB:82:AA:BF:45,enp7s0,enp7s0,10.137.132.2,4,HundredGigE0/0/0/7
node9-nic9-p1,p1,node9,net9,100,auto,,02:EF:CB:FB:4A:9D,enp7s0,enp7s0,10.141.1.2,6,HundredGigE0/0/0/5
node10-nic10-p1,p1,node10,net10,100,auto,,0E:CE:66:95:F6:1E,enp7s0,enp7s0,10.139.133.2,4,HundredGigE0/0/0/9



Time to print interfaces 832 seconds


In [4]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   FL_High1
Slice ID     9f7736a7-3107-4760-b192-95874bbeda87
Slice State  StableOK
Lease End    2025-05-07 05:52:02 +0000
-----------  ------------------------------------


In [107]:
%%capture
#Run this cell
#%%capture
try:
    for i, node in enumerate(slice.get_nodes()):
        node_iface=node.get_interface(network_name=network_names[i])
        node_IP_addr=available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_IP_addr, subnet=subnet)
        
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        _, _ = node.execute('sudo apt-get update')
        stdout, stderr = node.execute('sudo apt-get install net-tools')        
        stdout, stderr = node.execute(f'sudo ifconfig {node_iface.get_os_interface()} up')
        
except Exception as e:
    print(f"Exception: {e}")

In [108]:
for node in slice.get_nodes():
    try:
        stdout, stderr = node.execute("whoami")
        print(f"Node: {node}, Username: {stdout.strip()}")
    except Exception as e:
        print(f"Could not retrieve username for node {node}: {e}")

ubuntu
Node: -----------------  ------------------------------------------------------------------------------------------------------------------------------------------
ID                 d2fdca2e-4c74-43ef-93bf-2ec198b2f10d
Name               node1
Cores              24
RAM                64
Disk               500
Image              default_ubuntu_20
Image Type         qcow2
Host               clem-w2.fabric-testbed.net
Site               CLEM
Management IP      2620:103:a006:12:f816:3eff:fe7a:9137
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe7a:9137
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------, Username: ubuntu
ubuntu
Node: -----------------  ---------------------------------------------------------------------------------------

In [109]:
#Run this cell
# If the ping is successful, that means the nodes are connected properly if there is any error then may need to recreate the cluster or further have to look into it. 

try:
    node1=slice.get_node(name='node1')
    stdout,stderr=node1.execute(f' ping -c 3 10.135.6.2')
    print(stdout)
    print(stderr)
    
except Exception as e:
    print(f'Exception: {e}')
try:
    node13=slice.get_node(name='node13')
    stdout,stderr=node13.execute(f' ping -c 3 10.135.6.2')
    print(stdout)
    print(stderr)
    
except Exception as e:
    print(f'Exception: {e}') 

PING 10.135.6.2 (10.135.6.2) 56(84) bytes of data.

--- 10.135.6.2 ping statistics ---
3 packets transmitted, 0 received, 100% packet loss, time 2042ms

PING 10.135.6.2 (10.135.6.2) 56(84) bytes of data.

--- 10.135.6.2 ping statistics ---
3 packets transmitted, 0 received, 100% packet loss, time 2042ms



PING 10.135.6.2 (10.135.6.2) 56(84) bytes of data.

--- 10.135.6.2 ping statistics ---
3 packets transmitted, 0 received, 100% packet loss, time 2043ms

PING 10.135.6.2 (10.135.6.2) 56(84) bytes of data.

--- 10.135.6.2 ping statistics ---
3 packets transmitted, 0 received, 100% packet loss, time 2043ms





In [5]:
all_nodes = slice.get_nodes()
ips = []
for i, node in enumerate(all_nodes):
    ip_address = str(node.get_interface(network_name=network_names[i]).get_ip_addr())
    ips.append(ip_address)
print(ips)

['10.136.1.2', '10.133.134.2', '10.135.133.2', '10.129.139.2', '10.132.5.2', '10.137.4.2', '10.131.9.2', '10.137.132.2', '10.141.1.2', '10.139.133.2', '10.138.10.2', '10.139.3.2', '10.147.5.2', '10.146.1.2', '10.143.8.2', '10.132.13.2', '10.147.129.2', '10.134.140.2', '10.136.136.2', '10.138.11.2', '10.135.5.2']


In [9]:
public_key = ''
try:
    node21 = slice.get_node(name='node21')
    #Check if SSH key already exists, if not generate one
    stdout, stderr = node21.execute('test -f ~/.ssh/id_rsa.pub || ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N ""')
    #Now, safely read the public key
    stdout, stderr = node21.execute('cat ~/.ssh/id_rsa.pub')
    public_key = stdout.strip()
except Exception as e:
    print(f'Exception: {e}')


ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDJ98D5fL/+IQJMUuFQamcPoVHZNpYokhOtuHUBEqpZ9zeG4U0h1vrNfhy3JQlNnqmAZOgJu0Ts/sTLv8rRJguFYzdQnBLAXJm5A8gyvF7G+JhDg9LWBSvBDR/9/97p0wjzWQJ/eKlC86Aoj+raQIaybN8/iX/Sa11QDfmJpi/xR7Kxx45a9lgV4Ahkzz8EWRyblygAw0n9chJB2EtponjccFbm98JJrDJYcrMnkbzKoiRRQAsP3qdi3BRvDLUvsXuj65S7YcbyNLNXGEauZiLKBt5enxI/SDXOuYSYcrra8wKLdZqQXh+7fKIRzywHBtb9vCIxKnSE2pM5ZvDS/J0l30mKrPxJvnuC6pasBAjFKN+q1CCcl8ELCICGGlQ907puY7xgSlz9pscsBrcpgElKIqQydx0Nwe8NLR1wAQfy7CVG/a62rY/l6xmR/6CR8iIOmqtGOUTAWbCfTOF9x0kA+uCZYNt4T3sQA1ujPNYZhLE5bs8+7CFGv9XAmQaAkcb1L8q0nYZg8xkbiMb22d0P+L1o8WKw/Prwi8xqBS5zdlgutFju/0h2qrEZCJ35t/xnqrWIUOmuPIPsNj9H9TM0sz6XkKRVZDAp4+DmxMJshzd4kbPGhtvbjLTwaeLuYeYLvcwmSzGyUTgPUw/KNs+DzWUeoP9bmxfdt55gjipiRQ== ubuntu@node21


In [12]:
try:
    node21 = slice.get_node(name='node21')
    # Create the directory
    # stdout, stderr = node21.execute("mkdir -p ./fl_files")
    # stdout, stderr = node21.execute("sudo add-apt-repository universe")
    # stdout, stderr = node21.execute("sudo apt update")
    # stdout, stderr = node21.execute("sudo apt install python3-pip -y")
    # stdout, stderr = node21.execute("pip install datasets")
    # stdout, stderr = node21.execute("pip install Pillow")
    # stdout, stderr = node21.execute("sudo rm -rf /usr/lib/python3/dist-packages/OpenSSL")
    # stdout, stderr = node21.execute("sudo pip3 install pyopenssl")
    # stdout, stderr = node21.execute("sudo pip3 install pyopenssl --upgrade")
    # stdout, stderr = node21.execute("sudo pip3 install torch torchvision torchaudio")
    # stdout, stderr = node21.execute("sudo pip3 install flwr")
    # # Upload file (avoiding unpacking issue)
    # node21.upload_file('./fl_files/dataset.py', './fl_files/dataset.py')
    # node21.upload_file('./fl_files/dataset_2024.py', './fl_files/dataset_2024.py')
    # node21.upload_file('./fl_files/dataset_ISIC.py', './fl_files/dataset_ISIC.py')
    node21.upload_file('./fl_files/newServer.py', './fl_files/newServer.py')
    # node21.upload_file('./fl_files/statsServer.py', './fl_files/statsServer.py')
    # stdout, stderr = node21.execute('mkdir -p ./federated_ISIC_2019_data')
    # stdout, stderr = node21.execute("python3 ./fl_files/dataset_2024.py")
    # stdout, stderr = node21.execute("python3 ./fl_files/dataset_ISIC.py")
    # stdout, stderr = node21.execute("python3 ./fl_files/dataset.py")
    # #stdout, stderr = node21.execute("python3 ./fl_files/newServer.py --strategy fedavg --m ResNet18 --c 7 --lr 0.0005 --fraction_fit 0.5  --num_rounds 20")
    # stdout, stderr = node21.execute("mkdir -p ./results")
    # stdout, stderr = node21.execute("mkdir -p ./results/IID")
    # stdout, stderr = node21.execute("mkdir -p ./results/Non-IID")

except Exception as e:
    print(f'Exception: {e}')

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def run_client(client):
    try:
        try:
            print(f"Running on {client.get_name()}.")
            # stdout, stderr = client.execute(f'echo "{public_key}" >> ~/.ssh/authorized_keys')
            # # # Set correct permissions (important!)
            # stdout, stderr = client.execute('chmod 700 ~/.ssh')
            # stdout, stderr = client.execute('chmod 600 ~/.ssh/authorized_keys')
            # stdout, stderr = client.execute("mkdir -p ./fl_client")
            # stdout, stderr = client.execute("mkdir -p ./fl_client/datasets")
            # stdout, stderr = client.execute("mkdir -p ./fl_client/isic_datasets")
            # stdout, stderr = client.execute("mkdir -p ./fl_client/ham_datasets")
            # stdout, stderr = client.execute("mkdir -p ./results")
            # stdout, stderr = client.execute("mkdir -p ./results/IID")
            # stdout, stderr = client.execute("mkdir -p ./results/Non-IID")
            # stdout, stderr = client.execute("mkdir -p ./fl_client/skin_2024_datasets")
            client.upload_file('./fl_files/clientback.py','./fl_client/clientback.py')
            #client.upload_file('./fl_files/client.py','./fl_client/client.py')
            # stdout, stderr = client.execute("sudo add-apt-repository universe")
            # stdout, stderr = client.execute("sudo apt update")
            # stdout, stderr = client.execute("sudo apt install python3-pip -y")
            # stdout, stderr = client.execute("sudo pip3 install flwr")
            # stdout, stderr = client.execute("sudo rm -rf /usr/lib/python3/dist-packages/OpenSSL")
            # stdout, stderr = client.execute("sudo pip3 install pyopenssl")
            # stdout, stderr = client.execute("sudo pip3 install pyopenssl --upgrade")
            # stdout, stderr = client.execute("sudo pip3 install torch torchvision torchaudio")
            # stdout, stderr = node.execute(f"rm -rf ./fl_client/isic_datasets/client_{i+1}")
            # stdout, stderr = client.execute(f"rm -rf ./fl_client/isic_datasets/client_{i+1}"
            # stdout, stderr = client.execute(f"python3 ./fl_client/client.py -d isic_datasets -c 6")
            # stdout, stderr = client.execute(f"python3 ./fl_client/client.py -d ham_datasets -c 7 -i 10.135.6.2 -p 8080 -t IID -m ResNet50")
            stdout, stderr = client.execute(f"python3 ./fl_client/clientback.py -d isic_datasets -c 8 -i 10.135.5.2 -t IID -m ResNet50")
        except Exception as e:
            print(f'Exception: {e}')    
    except Exception as e:
        print(f"Client {client.get_name()} failed to fetch the file with exception: {e}")
 
try:
        # slice = fablib.get_slice(name=slice_name)
        client1, client2, client3,client4,client5, client6, client7,client8,client9, client10, client11,client12,client13, client14, client15,client16,client17, client18, client19,client20 = slice.get_node('node1'), slice.get_node('node2'), slice.get_node('node3'), slice.get_node('node4'), slice.get_node('node5'), slice.get_node('node6'), slice.get_node('node7'), slice.get_node('node8'), slice.get_node('node9'),slice.get_node('node10'), slice.get_node('node11'), slice.get_node('node12'),slice.get_node('node13'), slice.get_node('node14'), slice.get_node('node15'),slice.get_node('node16'), slice.get_node('node17'), slice.get_node('node18'),slice.get_node('node19'), slice.get_node('node20')
        slice_nodes = [client1, client2, client3,client4,client5, client6, client7,client8,client9, client10, client11,client12,client13, client14, client15,client16,client17, client18, client19,client20]
        with ThreadPoolExecutor(max_workers=len(slice_nodes)) as executor:
                executor.map(run_client, slice_nodes)

except Exception as e:
        print(f"Exception during parallel execution: {e}")

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Helper that SSH-executes a single command and returns timing info
def run_scp(node, cmd):
    start = time.time()
    out, err = node.execute(cmd)
    return cmd, time.time() - start, out, err

# In your main block:
try:
    node21 = slice.get_node(name='node21')
    for i, ip in enumerate(ips[:20]):
        print(f"\n→ Dispatching to client_{i+1} @ {ip}")
        # Build the three commands
        cmds = [
            f"scp -o StrictHostKeyChecking=no -r ./federated_ISIC_2019_data/client_{i+1}/* ubuntu@{ip}:/home/ubuntu/fl_client/isic_datasets/",
            f"scp -o StrictHostKeyChecking=no -r ./federated_skin_cancer/client_{i+1}/* ubuntu@{ip}:/home/ubuntu/fl_client/ham_datasets/",
            f"scp -o StrictHostKeyChecking=no -r ./federated_skin_2024/client_{i+1}/* ubuntu@{ip}:/home/ubuntu/fl_client/skin_2024_datasets/",
        ]

        # Use a thread pool of size 3 to run them in parallel
        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = [executor.submit(run_scp, node21, cmd) for cmd in cmds]

            # As each one completes, print its timing
            for future in as_completed(futures):
                cmd, elapsed, out, err = future.result()
                print(f"✅ `{cmd.split()[2]}` finished in {elapsed:.1f}s")
                if err:
                    print("   stderr:", err)

except Exception as e:
    print(f"Fatal error: {e}")


In [ ]:
for node in slice.get_nodes():
    try:
        stdout, stderr = node.execute("whoami")
        print(f"Node: {node}, Username: {stdout.strip()}")
    except Exception as e:
        print(f"Could not retrieve username for node {node}: {e}")

In [17]:
try:
    restore_folder = "./results/IID/"
    list_files_command = "ls ./results/IID/"
    # Check if directory exists and list files
    stdout, stderr = node21.execute(list_files_command)
    print(f"List files command output: {stdout}")
    if stderr:
        print(f"Error listing files: {stderr}")
    file_names = stdout.strip().split("\n")
    if not file_names or file_names == ['']:
        print("No files found in the directory.")
    else:
        print(f"Files found: {file_names}")

        # Loop through and download files
        for file in file_names:
            print(f"Processing file: {file}")
            result = node21.download_file(f"./fl_results/{file}",f"{restore_folder}/{file}")
            if result is None:
                print(f"{file} saved successfully.")
            else:
                print(f"Unexpected result from download_file: {result}")

except FileNotFoundError as fnf_error:
    print(f"FileNotFoundError: {fnf_error}")
except Exception as e:
    print(f"Exception: {e}")

isic_2019_fedavg_evaluation.csv
isic_2019_fedsgd_evaluation.csv
List files command output: isic_2019_fedavg_evaluation.csv
isic_2019_fedsgd_evaluation.csv

Files found: ['isic_2019_fedavg_evaluation.csv', 'isic_2019_fedsgd_evaluation.csv']
Processing file: isic_2019_fedavg_evaluation.csv
isic_2019_fedavg_evaluation.csv saved successfully.
Processing file: isic_2019_fedsgd_evaluation.csv
isic_2019_fedsgd_evaluation.csv saved successfully.


In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Directory containing the evaluation files
data_dir = "./fl_results"

# Find all .xlsx evaluation files
files = [
    f for f in os.listdir(data_dir)
    if f.endswith("_evaluation.xlsx")
]

# Load and concatenate all files with dataset and method columns
df_list = []
for f in files:
    path = os.path.join(data_dir, f)
    df = pd.read_excel(path)
    # Derive dataset and method from filename
    base = f[:-len("_evaluation.xlsx")]
    parts = base.split("_")
    if len(parts) < 2:
        # Skip unexpected filenames
        continue
    method = parts[-1]
    dataset = "_".join(parts[:-1])
    df["dataset"] = dataset
    df["method"] = method
    df_list.append(df)

df_all = pd.concat(df_list, ignore_index=True)

# Metrics to plot
metrics = ["loss", "accuracy", "training_time"]

# For each dataset, plot each metric vs. round
for dataset, group in df_all.groupby("dataset"):
    for metric in metrics:
        plt.figure()
        for method, sub in group.groupby("method"):
            plt.plot(sub["round"], sub[metric], marker='o', label=method)
        plt.xlabel("Round")
        plt.ylabel(metric.replace("_", " ").title())
        plt.title(f"{dataset} - {metric.replace('_', ' ').title()} vs Round")
        plt.legend(title="Method")
        plt.grid(True)
        plt.show()


ValueError: No objects to concatenate

In [17]:
# Start Cassandra on each node
import time
for i, node in enumerate(all_nodes):
    node_id = node.get_name()
    if i!= 20:
         stdout, stderr = node.execute("rm -rf ./dataset")

In [27]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")


Retry: 3, Time: 115 sec


ID,9f7736a7-3107-4760-b192-95874bbeda87
Name,FL_High1
Lease Expiration (UTC),2025-05-07 05:52:02 +0000
Lease Start (UTC),2025-04-23 23:27:28 +0000
Project ID,b3cffedd-ddb4-43ee-b57d-459b768e14ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
d2fdca2e-4c74-43ef-93bf-2ec198b2f10d,node1,24,64,500,default_ubuntu_20,qcow2,clem-w2.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe7a:9137,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe7a:9137,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8d5f9b42-d820-4f43-9190-8be7d5f81638,node10,24,64,500,default_ubuntu_20,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fe19:418d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe19:418d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
38028455-a1c3-40b7-8f22-ceaa30486348,node11,24,64,500,default_ubuntu_20,qcow2,kans-w2.fabric-testbed.net,KANS,ubuntu,2001:400:a100:3060:f816:3eff:fe46:778d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fe46:778d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c7e05851-f5e7-435d-8ef5-de9c74994b8b,node12,24,64,500,default_ubuntu_20,qcow2,seat-w2.fabric-testbed.net,SEAT,ubuntu,2001:400:a100:3080:f816:3eff:fe62:d4ea,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe62:d4ea,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
93884021-683e-410c-977c-e7b2f39c53cd,node13,24,64,500,default_ubuntu_20,qcow2,hawi-w1.fabric-testbed.net,HAWI,ubuntu,2607:f278:1:202:f816:3eff:fea4:e2af,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f278:1:202:f816:3eff:fea4:e2af,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7ee24723-6d0e-4848-a6f6-e49bfda8cf0e,node14,24,64,500,default_ubuntu_20,qcow2,toky-w2.fabric-testbed.net,TOKY,ubuntu,133.69.160.18,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.18,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
31419306-4cfb-472a-a496-d0c1cb5b0317,node15,24,64,500,default_ubuntu_20,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe14:5147,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe14:5147,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2469a567-2e9a-47bb-a188-3fcf0fb7643c,node16,24,64,500,default_ubuntu_20,qcow2,utah-w5.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fea2:3e1e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fea2:3e1e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
69070002-7adc-4a45-b77f-d46499b81049,node17,24,64,500,default_ubuntu_20,qcow2,eduky-w7.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe2e:d4b0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe2e:d4b0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1e15cd6a-ffce-4cac-8467-f260bd42dd35,node18,24,64,500,default_ubuntu_20,qcow2,ucsd-w5.fabric-testbed.net,UCSD,ubuntu,132.249.252.143,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.143,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
2b1bdcd1-64d7-4cd6-84e4-525c2a04298c,net1,L3,FABNetv4,CLEM,10.136.1.0/24,10.136.1.1,Active,
49c00b81-62e6-4074-bc7f-e4126fa66b28,net10,L3,FABNetv4,PRIN,10.139.133.0/24,10.139.133.1,Active,
f1486016-7459-4f40-96c4-645c66ce9801,net11,L3,FABNetv4,KANS,10.138.10.0/24,10.138.10.1,Active,
1235c930-106d-4330-9ff6-d3ab7c216bab,net12,L3,FABNetv4,SEAT,10.139.3.0/24,10.139.3.1,Active,
94aebaa6-811b-4a8b-8f8f-be52ace534e7,net13,L3,FABNetv4,HAWI,10.147.5.0/24,10.147.5.1,Active,
21b5a742-c419-43e5-9a99-d36bc0284642,net14,L3,FABNetv4,TOKY,10.146.1.0/24,10.146.1.1,Active,
3f5df975-270d-484b-89a1-29ec2da07ef1,net15,L3,FABNetv4,CERN,10.143.8.0/24,10.143.8.1,Active,
7db5d96d-ba8e-4386-bfc6-7dfe3ac43fd2,net16,L3,FABNetv4,UTAH,10.132.13.0/24,10.132.13.1,Active,
9683856a-1e53-472a-9a4b-2446f4eec211,net17,L3,FABNetv4,EDUKY,10.147.129.0/24,10.147.129.1,Active,
745a200d-de5d-4165-8b92-d8fa01ba2c12,net18,L3,FABNetv4,UCSD,10.134.140.0/24,10.134.140.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
node1-nic1-p1,p1,node1,net1,100,auto,,02:DA:D0:DE:84:4D,enp7s0,enp7s0,10.136.1.2,4,HundredGigE0/0/0/9
node2-nic2-p1,p1,node2,net2,100,auto,,0E:0A:CD:51:48:0A,enp7s0,enp7s0,10.133.134.2,6,HundredGigE0/0/0/5
node3-nic3-p1,p1,node3,net3,100,auto,,0E:C7:B8:E0:CD:11,enp7s0,enp7s0,10.135.133.2,6,HundredGigE0/0/0/5
node4-nic4-p1,p1,node4,net4,100,auto,,22:16:D2:54:E0:CB,enp7s0,enp7s0,10.129.139.2,6,HundredGigE0/0/0/5
node5-nic5-p1,p1,node5,net5,100,auto,,16:4A:C8:88:50:28,enp7s0,enp7s0,10.132.5.2,4,HundredGigE0/0/0/11
node6-nic6-p1,p1,node6,net6,100,auto,,06:F9:BA:9E:5A:38,enp6s0,enp6s0,10.137.4.2,4,HundredGigE0/0/0/7
node7-nic7-p1,p1,node7,net7,100,auto,,0E:06:73:C1:D9:AE,enp7s0,enp7s0,10.131.9.2,6,HundredGigE0/0/0/5
node8-nic8-p1,p1,node8,net8,100,auto,,06:AB:82:AA:BF:45,enp7s0,enp7s0,10.137.132.2,4,HundredGigE0/0/0/7
node9-nic9-p1,p1,node9,net9,100,auto,,02:EF:CB:FB:4A:9D,enp7s0,enp7s0,10.141.1.2,6,HundredGigE0/0/0/5
node10-nic10-p1,p1,node10,net10,100,auto,,0E:CE:66:95:F6:1E,enp7s0,enp7s0,10.139.133.2,4,HundredGigE0/0/0/9



Time to print interfaces 120 seconds


In [28]:
try:
    print(slice_name)
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End (UTC)        : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

FL_High1
Lease End (UTC)        : 2025-05-07 05:52:02 +0000


In [92]:
fablib.delete_slice(slice_name)